This notebook presents an application of the [pydiva2D](./pydiva2d.py) module to perform a 2D Diva analysis and then generate figures for the different input and output using the [Folium](https://pypi.python.org/pypi/folium) module.<br>

**Note:** since this module accesses various javascript libraries hosted on CDNs, an internet connection is needed to have a working example directly.<br>
If you're familiar with leaflet, a workaround consist in editing the html file generated with [Folium](https://github.com/python-visualization/folium), and replace the URLs to the libraries with local paths.

In [1]:
import os
import logging
import numpy as np
import matplotlib.pyplot as plt
import folium
from folium import plugins
import json
import geojson
import subprocess
from importlib import reload
from mpl_toolkits.basemap import Basemap 

In [2]:
import sys
sys.path.insert(0, "../")
import pydiva2d

We set up the logger so that only *info* messages are displayed on screen.

In [3]:
logger = logging.getLogger('diva2D')
logger.setLevel(logging.DEBUG)
pydiva2d.logger.setLevel(logging.DEBUG)

# Prepare input files and directories

In this example the input files are already created and can be found in the *data* directory.

In [4]:
datadir = './data/'
datafile = os.path.join(datadir, 'MLD1.dat')
coastfile = os.path.join(datadir, 'coast.cont')
paramfile = os.path.join(datadir, 'param.par')

We also create a directory *html* where we will save the resulting `Leaflet` maps.

In [5]:
htmldir = './html/'
if os.path.exists(htmldir):
    logger.debug("Directory {0} already exists".format(htmldir))
else:
    logger.debug("Creating directory {0}".format(htmldir))
    os.makedirs(htmldir)

## Create paths and file names

We create the paths for the Diva directories and files:

In [6]:
divadir = "/home/ctroupin/Software/DIVA/DIVA-diva-4.7.1"
DivaDirs = pydiva2d.DivaDirectories(divadir)
DivaFiles = pydiva2d.Diva2Dfiles(DivaDirs.diva2d)

## Read input files

Let's read the information from the input files.<br>
For each input file, we first creat an object (i.e., **Data**, **Contour** and **Param**) that will allow us to perform operations such as 
* reading, 
* writing, 
* plotting, 
* getting a description.

In [7]:
Data = pydiva2d.Diva2DData()
Data.read_from("../data/MLD1.dat")
Contour = pydiva2d.Diva2DContours()
Contour.read_from("../data/coast.cont")
Param = pydiva2d.Diva2DParameters()
Param.read_from("../data/param.par")

# Make the analysis

We write the inputs to the corresponding files using the *write_to* functions available for each type of input:

In [8]:
Data.write_to(DivaFiles.data)
Param.write_to(DivaFiles.parameter)
Contour.write_to(DivaFiles.contour)

## Generate the mesh

We simply call the `divamesh` script:

In [9]:
subprocess.Popen("./divamesh", cwd=DivaDirs.diva2d, shell=True)

and we create a new **Mesh** object using the information from the 2 files generated by the mesh generation:

In [10]:
Mesh = pydiva2d.Diva2DMesh()
Mesh.read_from(DivaFiles.mesh, DivaFiles.meshtopo)

## Perform the analysis

We call the `divacalc` script:

In [11]:
subprocess.Popen("./divacalc", cwd=DivaDirs.diva2d, shell=True)

and load the results in a new object **Analysis**:

In [12]:
Analysis = pydiva2d.Diva2DResults(DivaFiles.result)

2017-04-27 15:43:54,746 - pydiva2d - WARNING - No error field in the netCDF file (will return NaN's)


# Create the maps

In this example the plots are created using the [Folium](https://github.com/python-visualization/folium) module.<br>
In each step of the map creation, we will save a html file containg the different elements.

## Initialise the projection

We will center the map on the region specified by the parameters.<br>
The zoom level (zoom_start) has to be adapted according to the extension of the region.

In [13]:
divamap = folium.Map(location=[.5 * (Param.yori + Param.yend) , .5 * (Param.xori + Param.xend)], 
                     min_lat=Param.yori, max_lat=Param.yend, 
                     min_lon=Param.xori, max_lon=Param.xend,
                     zoom_start=7)
divamap.save(os.path.join(htmldir, 'basemap.html'))

## Data points

We place a circle for each data location.

In [14]:
for lat, lon in zip(Data.y, Data.x):
    #datapoints = folium.CircleMarker([Data.y, Data.x])
    folium.CircleMarker(location=[lat, lon], radius=5, color='green').add_to(divamap)
    #divamap.add_child(datapoints)
divamap.save(os.path.join(htmldir, 'datapoints.html'))

## Contours

The different contours are read and represented as *Polylines*.

In [15]:
for lons, lats in zip(Contour.x, Contour.y):
    # Note that coordinates are defined as (lat, lon)
    coords = zip(lats, lons)
    folium.PolyLine(locations=coords, color='blue', weight=5, opacity=0.75).add_to(divamap)
divamap.save(os.path.join(htmldir, 'contours.html'))

## Analysis

The procedure works in two steps:
1. create a figure (png),
2. add the figure as an overlay to the map.

### Generation of a figure

We want to prepare a figure that has no border, axis, title, ...<br>
It is also necessary to specify the projection ([epsg 3857](http://spatialreference.org/ref/sr-org/6864/)) in order to fit with the map background.

In [16]:
resultfigname = 'analysedfield.png'
m = Basemap(llcrnrlon=Param.xori, llcrnrlat=Param.yori,
            urcrnrlon=Param.xend, urcrnrlat=Param.yend, resolution = 'l', epsg=3857)
lon, lat = np.meshgrid(Analysis.x, Analysis.y)
llon, llat = m(lon, lat)
fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
Analysis.add_to_plot(field='analysis', m=m)
f1 = plt.gca()
f1.axes.get_xaxis().set_ticks([])
f1.axes.get_yaxis().set_ticks([])
# plt.show()
plt.savefig(os.path.join(htmldir, resultfigname), transparent=True, bbox_inches='tight', pad_inches=0)
plt.close()

/usr/local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3413: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
/usr/local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3422: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)


### Create overlay

Now the figure is created, it can be added as an overlay to the map.<br>
To do so we use the [ImageOverlay](https://github.com/python-visualization/folium/blob/master/folium/plugins/image_overlay.py) method. Note that one can also use a numpy array as input, but here we prefer to first generate the figure in order to have more control on its aspect.

In [17]:
folium.plugins.ImageOverlay(resultfigname,
                            bounds=[[Param.yori, Param.xori], [Param.yend, Param.xend]], 
                            opacity=.8
                           ).add_to(divamap)
divamap.save(os.path.join(htmldir, 'results.html'))

## Finite-element mesh

We follow the same procedure as for the analysed field:
* generate a figure,
* add it as an overlay.

In [18]:
meshfigname = 'mesh.png'
m = Basemap(llcrnrlon=Param.xori, llcrnrlat=Param.yori,
            urcrnrlon=Param.xend, urcrnrlat=Param.yend, resolution = 'l', epsg=3857)
fig = plt.figure(frameon=False)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
Mesh.add_to_plot(m=m, linewidth=.2, color='k')
f1 = plt.gca()
f1.axes.get_xaxis().set_ticks([])
f1.axes.get_yaxis().set_ticks([])
# plt.show()
plt.savefig(os.path.join(htmldir, meshfigname), transparent=True, bbox_inches='tight', pad_inches=0)
plt.close()

/usr/local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3260: MatplotlibDeprecationWarning: The ishold function was deprecated in version 2.0.
  b = ax.ishold()
/usr/local/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:3269: MatplotlibDeprecationWarning: axes.hold is deprecated.
    See the API Changes document (http://matplotlib.org/api/api_changes.html)
    for more details.
  ax.hold(b)


In [19]:
folium.plugins.ImageOverlay(meshfigname,
                            bounds=[[Param.yori, Param.xori], [Param.yend, Param.xend]],
                           ).add_to(divamap)
divamap.save(os.path.join(htmldir, 'mesh.html'))

# Visualisation

We have create one html file per level of plot: 
* [data points](./html/datapoints.html), 
* [contours](contours.html), 
* [analysed fields](results.html)
* [mesh](./html/mesh.html).

One can easily combine and customise the different elements of the plot to obtained the desired Leaflet map.